<a href="https://colab.research.google.com/github/franseal/project_X/blob/master/%E5%B7%9D%E6%9F%B3%E7%94%9F%E6%88%90%E5%99%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#install

In [0]:
import numpy as np

In [2]:
!pip install janome

    100% |████████████████████████████████| 20.7MB 2.4MB/s 


In [3]:
!apt-get -q -y install swig
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3

Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  swig3.0
Suggested packages:
  swig-doc swig-examples swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  swig swig3.0
0 upgraded, 2 newly installed, 0 to remove and 13 not upgraded.
Need to get 1,100 kB of archives.
After this operation, 5,822 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig3.0 amd64 3.0.12-1 [1,094 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig amd64 3.0.12-1 [6,460 B]
Fetched 1,100 kB in 0s (10.1 MB/s)
Selecting previously unselected package swig3.0.
(Reading database ... 110851 files and directories currently installed.)
Preparing to unpack .../swig3.0_3.0.12-1_amd64.deb ...
Unpacking swig3.0 (3.0.12-1) ...
Selecting previously unselected package swig.
Preparing to unpack .../swig_3.0.12-1_amd64.deb ...
Unpacking swig (3.0.12-

In [4]:
!pip install chainer
!pip install cupy-cuda92

In [5]:
!pip install tqdm

In [6]:
!wget http://www.cl.ecei.tohoku.ac.jp/nlp100/data/neko.txt

--2019-01-10 06:25:47--  http://www.cl.ecei.tohoku.ac.jp/nlp100/data/neko.txt
Resolving www.cl.ecei.tohoku.ac.jp (www.cl.ecei.tohoku.ac.jp)... 130.34.192.83
Connecting to www.cl.ecei.tohoku.ac.jp (www.cl.ecei.tohoku.ac.jp)|130.34.192.83|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 965825 (943K) [text/plain]
Saving to: ‘neko.txt’

neko.txt            100%[===================>] 943.19K   687KB/s    in 1.4s    

2019-01-10 06:25:49 (687 KB/s) - ‘neko.txt’ saved [965825/965825]



#wakati

In [0]:
from tqdm import tqdm
import MeCab
import numpy as np

In [0]:
mecab = MeCab.Tagger("-Ochasen")

In [0]:
from google.colab import files
uploaded = files.upload()

Saving natsume_c.txt to natsume_c.txt


In [0]:
text = open("neko.txt").readlines()

In [10]:
wakati = []
for t in tqdm(text):
  for row in mecab.parse(t).split("\n"):
    row = row.split("\t")
    if len(row) > 3:
      wakati.append(row[0])

100%|██████████| 9964/9964 [00:00<00:00, 15774.57it/s]


In [0]:
wakati = np.array(wakati)

In [12]:
from collections import defaultdict
word2id = defaultdict(lambda: len(word2id))
for w in wakati:
  word2id[w]
  
len(word2id)

13584

In [13]:
id2word = {i:w for w,i in word2id.items()}
len(id2word)

13584

In [14]:
_wakati = wakati[1:]
_wakati = _wakati[_wakati != "\u3000"]
sentences = []
st = 0
for i,w in enumerate(_wakati):
  if w == "。":
    sentences.append(_wakati[st:i+1])
    st = i + 1
    
print(len(sentences))
sentences[0]

7486


array(['吾輩', 'は', '猫', 'で', 'ある', '。'], dtype='<U25')

In [16]:
def wlist_to_id(sen):
  return [word2id[w] for w in sen]

train_data = [wlist_to_id(sen) for sen in sentences]
train_data[0]

[2, 3, 4, 5, 6, 7]

#network

In [0]:
import chainer
from chainer.backends import cuda
from chainer import Function, gradient_check, report, training, utils, Variable
from chainer import datasets, iterators, optimizers, serializers
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L
from chainer.training import extensions
import cupy as cp
import random
import copy

In [0]:
class SentenceGenerator(Chain):
  def __init__(self, vocab_size, n_embed=100, n_mid=400):
    super().__init__()
    with super().init_scope():
      self.embed = L.EmbedID(vocab_size, n_embed)
      self.lstm = L.NStepLSTM(n_layers=2, in_size=n_embed, out_size=n_mid, dropout=0)
      self.out = L.Linear(n_mid, vocab_size)
      
  def __call__(self, list_of_sentences):
    xs = [self.embed(Variable(sen[ :-1])) for sen in list_of_sentences]
    ts = [           Variable(sen[1:  ])  for sen in list_of_sentences]
    
    hs, cs, ys = self.lstm(None, None, xs)
    loss = 0.0
    for y, t in zip(ys,ts):
      z = self.out(y)
      loss += F.softmax_cross_entropy(z, t)
    return loss
  
  def predict(self, prefix):
    xs = [self.embed(Variable(prefix))]
    hs, cs, ys = self.lstm(None, None, xs)
    y = ys[0]
    z = self.out(y)
    return F.softmax(z)[-1].data

In [19]:
model = SentenceGenerator(vocab_size=len(word2id))
model.to_gpu(0)

optimizer = optimizers.Adam()
optimizer.setup(model)

#学習

In [28]:
from tqdm import trange

batchsize = 128
for epoch in range(10): #学習回数
  shuffled = np.random.permutation(len(train_data))
  sum_loss = 0.0
  n=0
  
  for i in trange(0, len(train_data), batchsize):
    ids = shuffled[i:i+batchsize]
    xs = [cp.array(train_data[i]) for i in ids]
    
    model.cleargrads()
    loss = model(xs)
    loss.backward()
    optimizer.update()
    
    sum_loss += loss.data
    #print(loss.data)
    n += len(ids)
    
  print("Epoch {} : loss {}".format(epoch, sum_loss / n))

  0%|          | 0/59 [00:00<?, ?it/s]

Epoch 0 : loss 6.0179057


  0%|          | 0/59 [00:00<?, ?it/s]

Epoch 1 : loss 5.578831


  0%|          | 0/59 [00:00<?, ?it/s]

...

Epoch 7 : loss 4.4147205


  0%|          | 0/59 [00:00<?, ?it/s]

Epoch 8 : loss 4.2728195


100%|██████████| 59/59 [00:36<00:00,  1.92it/s]

Epoch 9 : loss 4.132431


#生成
引数はsen, 最初の生成で上位いくつワードを取ってくるか, 以降の生成回数

In [0]:
from janome.tokenizer import Tokenizer
import re

In [0]:
def first(sen, seisei):
  keeplist = []
  result = []
  
  for i in range(1):
    with chainer.using_config("train", False):
      pr = model.predict(cp.array(sen))
    pr = chainer.cuda.to_cpu(pr)
    pr = np.argsort(pr)[::-1]
    
    for j in range(seisei):
      sen1 = copy.deepcopy(sen)
      sen1.append(int(pr[j]))
      keeplist.append(sen1)
      
  t = Tokenizer()
  wordcount = 0
  word = ""
  for i in range(len(keeplist)):
    s = "".join([id2word[i] for i in keeplist[i]])
    for token in t.tokenize(s, stream=True):
      match = re.search(r"[ャュョヮ]", token.reading)
      wordcount += (len(token.reading))
      if match:
        wordcount -= (len(re.findall(r"[ャュョヮ]", token.reading)))
    if wordcount == 5:
      word = (token.part_of_speech.split(",")[0])
      if word == "助詞":
        result.append(keeplist[i])
    wordcount = 0
    word = ""
    
  return result

In [0]:
def second(sen, seisei, repeat):
  keeplist = []
  result = []
  
  for i in range(1):
    with chainer.using_config("train", False):
      pr = model.predict(cp.array(sen))
    pr = chainer.cuda.to_cpu(pr)
    pr = np.argsort(pr)[::-1]
    
    for j in range(seisei):
      sen1 = copy.deepcopy(sen)
      sen1.append(int(pr[j]))
      keeplist.append(sen1)
      
  for i in range(repeat):
    with chainer.using_config("train", False):
      sen = copy.deepcopy(keeplist[i])
      pr = model.predict(cp.array(sen))
    pr = chainer.cuda.to_cpu(pr)
    pr = np.argsort(pr)[::-1]
    
    for j in range(seisei):
      sen1 = copy.deepcopy(sen)
      sen1.append(int(pr[j]))
      keeplist.append(sen1)
      
  t = Tokenizer()
  wordcount = 0
  word = ""
  for i in range(len(keeplist)):
    s = "".join([id2word[i] for i in keeplist[i]])
    for token in t.tokenize(s, stream=True):
      match = re.search(r"[ャュョヮ]", token.reading)
      wordcount += (len(token.reading))
      if match:
        wordcount -= (len(re.findall(r"[ャュョヮ]", token.reading)))
    if wordcount == 12:
      word = (token.part_of_speech.split(",")[0])
      if word == "助詞":
        result.append(keeplist[i])
    wordcount = 0
    word = ""
    
  return result

In [0]:
def third(sen, seisei, repeat):
  keeplist = []
  result = []
  
  for i in range(1):
    with chainer.using_config("train", False):
      pr = model.predict(cp.array(sen))
    pr = chainer.cuda.to_cpu(pr)
    pr = np.argsort(pr)[::-1]
    
    for j in range(seisei):
      sen1 = copy.deepcopy(sen)
      sen1.append(int(pr[j]))
      keeplist.append(sen1)
      
  for i in range(repeat):
    with chainer.using_config("train", False):
      sen = copy.deepcopy(keeplist[i])
      pr = model.predict(cp.array(sen))
    pr = chainer.cuda.to_cpu(pr)
    pr = np.argsort(pr)[::-1]
    
    for j in range(seisei):
      sen1 = copy.deepcopy(sen)
      sen1.append(int(pr[j]))
      keeplist.append(sen1)
      
  t = Tokenizer()
  wordcount = 0
  word = ""
  for i in range(len(keeplist)):
    s = "".join([id2word[i] for i in keeplist[i]])
    for token in t.tokenize(s, stream=True):
      match = re.search(r"[ャュョヮ]", token.reading)
      wordcount += (len(token.reading))
      if match:
        wordcount -= (len(re.findall(r"[ャュョヮ]", token.reading)))
    if wordcount == 17:
      word = (token.part_of_speech.split(",")[0])
      #if word == "助詞":
      result.append(keeplist[i])
    wordcount = 0
    word = ""
    
  return result

In [29]:
kari = []
res2 = []
res3 = []

sen = [word2id["人間"]]
res = first(sen, 80)
print(res)

for i in range(len(res)):
  sen = res[i]
  kari = second(sen, 10, 20)
  for j in range(len(kari)):
    res2.append(kari[j])
print(res2)

for i in range(len(res2)):
  sen = res2[i]
  kari = third(sen, 10, 20)
  for j in range(len(kari)):
    res3.append(kari[j])
print(res3)

for i in range(len(res3)):
  s = "".join([id2word[i] for i in res3[i]])
  print(s)

[[35, 3], [35, 55], [35, 76], [35, 17], [35, 38], [35, 66], [35, 5], [35, 43], [35, 49], [35, 204], [35, 14], [35, 1106], [35, 558], [35, 102], [35, 28]]
[[35, 3, 54, 1494, 68], [35, 3, 54, 820, 68], [35, 76, 3, 599, 3347], [35, 76, 3, 2, 68], [35, 76, 3, 2, 321], [35, 76, 3, 35, 21], [35, 66, 54, 820, 68], [35, 43, 3, 2, 68], [35, 43, 3, 2, 321], [35, 14, 43, 95, 372]]
[[35, 3, 54, 1494, 68, 148, 2], [35, 3, 54, 1494, 68, 2, 55], [35, 3, 54, 1494, 68, 2, 3], [35, 3, 54, 1494, 68, 2, 76], [35, 3, 54, 1494, 68, 2, 38], [35, 3, 54, 1494, 68, 2, 17], [35, 3, 54, 1494, 68, 2, 66], [35, 3, 54, 1494, 68, 2, 5], [35, 3, 54, 1494, 68, 2, 43], [35, 3, 54, 1494, 68, 54, 194], [35, 3, 54, 1494, 68, 54, 820], [35, 3, 54, 1494, 68, 54, 369], [35, 3, 54, 1494, 68, 54, 1494], [35, 3, 54, 1494, 68, 63, 820], [35, 3, 54, 1494, 68, 63, 194], [35, 3, 54, 1494, 68, 63, 2912], [35, 3, 54, 1494, 68, 369, 68], [35, 3, 54, 1494, 68, 148, 63, 81], [35, 3, 54, 1494, 68, 148, 63, 86], [35, 3, 54, 1494, 68, 148, 